In [1]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
	dtype = torch.cuda.FloatTensor
else:
	dtype = torch.FloatTensor


text_file_19982012 = '20042012.pkl'
text_file_20142018 = '20142019.pkl'
text_file_20122014 = '20122014.pkl'

#Defines the amount of hours later we want to forecast the Dst
forecasthour = 4

#maximum total amount of epochs
#do not take this tooo low because when error on CV will not increase (when locl minimum, when learning rate too small)
num_epochs = 300

input_size = 6
output_size = 1

#hyperparameters from grid
num_layers = 1
hidden_layer_size = 32
learning_rate = 0.0001
lookbackhour = 30
batch_size = 256
dropout = 0

In [2]:


# load in the scaler data 1998-2012

datapanda = pd.read_pickle(text_file_19982012)

#Quentifify total number of parameters - timestamps
NrParam = datapanda.values[0].shape[0] - 3
#get all the input parameters including Dst (without timestamps)
inputdata = datapanda.loc[:,3:(2+NrParam)].values
#get the Dst output
outputdata = datapanda.loc[:,(2+NrParam)].values
outputdata = outputdata.reshape(-1, 1)

#use the preprocessing tool of sklearn to rescale data between -1 and 1
inputscaler = preprocessing.StandardScaler()
inputscaler.fit(inputdata)
traininginput = inputdata
trainingoutput = outputdata
#save scaler to later inverse the scaled result
scaler = preprocessing.StandardScaler()
scaler.fit(outputdata)

# load in the cv data 2012-2014

datapanda = pd.read_pickle(text_file_20122014)

#Quentifify total number of parameters - timestamps
NrParam = datapanda.values[0].shape[0] - 3
#get all the input parameters including Dst (without timestamps)
CVinput = datapanda.loc[:,3:(2+NrParam)].values
#get the Dst output
CVoutput = datapanda.loc[:,(2+NrParam)].values
CVoutput = CVoutput.reshape(-1, 1)

# load in the test data 2014-2019

datapanda = pd.read_pickle(text_file_20142018)

#Quentifify total number of parameters - timestamps
NrParam = datapanda.values[0].shape[0] - 3
#get all the input parameters including Dst (without timestamps)
testinput = datapanda.loc[:,3:(2+NrParam)].values
#get the Dst output
testoutput = datapanda.loc[:,(2+NrParam)].values
testoutput = testoutput.reshape(-1, 1)

realtestoutput = testoutput
realtestinput = testinput


traininginput = inputscaler.transform(traininginput)
CVinput = inputscaler.transform(CVinput)
testinput = inputscaler.transform(testinput)
trainingoutput = scaler.transform(trainingoutput)
CVoutput = scaler.transform(CVoutput)
testoutput = scaler.transform(testoutput)


def create_dataset(datasetinput, datasetoutput, look_back, forecast):
	dataX, dataY = [],[]
	for i in range(len(datasetinput) - int(look_back) - int(forecast) - 1):
		x = datasetinput[i:(i+int(look_back))]
		dataX.append(x)
		y = datasetoutput[(i+int(forecast)):(i+int(look_back)+int(forecast))]
		dataY.append(y)
	return np.array(dataX), np.array(dataY), len(dataY)

def prepare_data(inputdata, outputdata, lookback, forecast, batchsize):
	lookback = int(lookback)
	forecast = int(forecast)
	batchsize = int(batchsize)

	numberofdata = len(inputdata)
	number_of_batches = (numberofdata // batchsize)

	#create a list where we will put all the batches in
	Xlist = []
	Ylist = []

	dataX, dataY, numberdummy = create_dataset(inputdata, outputdata, lookback, forecast)

	for b in range(0,number_of_batches-1):
		Xbatch2 = dataX[(b*batchsize):((b+1)*batchsize)]
		Ybatch2 = dataY[(b*batchsize):((b+1)*batchsize)]
		Xtensor2 = torch.Tensor(Xbatch2).type(dtype)
		#transpose because we want shape (seq length, batchsize, number of features)
		Xtensor2 = Xtensor2.transpose(1,0)
		Ytensor2 = torch.Tensor(Ybatch2).type(dtype)
		Ytensor2 = Ytensor2.transpose(1,0)
		#add tensor batches to lists of all batches
		Xlist.append(Xtensor2)
		Ylist.append(Ytensor2)

	return Xlist, Ylist


/home/jaripeeperkorn/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jaripeeperkorn/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jaripeeperkorn/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jaripeeperkorn/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [3]:
#We use MSE loss
loss_fn = nn.MSELoss()



testX, testY = prepare_data(testinput, testoutput, lookbackhour, forecasthour, batch_size)

#create unscaled versions to use to test later
textXunsc, textYunsc = prepare_data(realtestinput, realtestoutput, lookbackhour, forecasthour, 1)


In [4]:
# Here we define our model as a class
class lstm(nn.Module):
	def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout):
		super(lstm, self).__init__()
		self.input_dim = input_dim
		self.hidden_dim = hidden_dim
		self.num_layers = num_layers
		self.dropout = dropout

		# Define the LSTM layer(s)
		self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, dropout = self.dropout)

		# Define the output layer
		self.linear = nn.Linear(self.hidden_dim, output_dim)

	def forward(self, input):
		batch_size = input.size(1)
		hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).type(dtype), torch.zeros(self.num_layers, batch_size, self.hidden_dim).type(dtype)
		# Forward pass through LSTM layer
		# shape of lstm_out: [input sequence length, batch_size, hidden_dim (features)]
		# shape of self.hidden: (a, b), where a and b both have shape (num_layers, batch_size, hidden_dim).
		lstm_out, hidden = self.lstm(input.view(len(input), batch_size, -1))
		y_pred = self.linear(lstm_out)
		return y_pred


In [ ]:
model = torch.load('model5.pth')
model.eval()

lstm(
  (lstm): LSTM(6, 64, num_layers=2)
  (linear): Linear(in_features=64, out_features=1, bias=True)
)

In [ ]:
def prepare_data2(inputdata, outputdata, lookback, forecast, batchsize):
	lookback = int(lookback)
	forecast = int(forecast)
	batchsize = int(batchsize)

	numberofdata = len(inputdata)
	number_of_batches = (numberofdata // batchsize) - lookback - forecast

	#create a list where we will put all the batches in
	Xlist = []
	Ylist = []

	dataX, dataY, numberdummy = create_dataset(inputdata, outputdata, lookback, forecast)

	for b in range(0,number_of_batches-1):
		Xbatch2 = dataX[(b*batchsize):((b+1)*batchsize)]
		Ybatch2 = dataY[(b*batchsize):((b+1)*batchsize)]
		Xtensor2 = torch.Tensor(Xbatch2).type(dtype)
		#transpose because we want shape (seq length, batchsize, number of features)
		Xtensor2 = Xtensor2.transpose(1,0)
		Ytensor2 = torch.Tensor(Ybatch2).type(dtype)
		Ytensor2 = Ytensor2.transpose(1,0)
		#add tensor batches to lists of all batches
		Xlist.append(Xtensor2)
		Ylist.append(Ytensor2)

	return Xlist, Ylist


In [ ]:
testX, testY = prepare_data2(testinput, testoutput, lookbackhour, forecasthour, 1)

#create unscaled versions to use to test later
testXunsc, testYunsc = prepare_data2(realtestinput, realtestoutput, lookbackhour, forecasthour, 1)

print(len(testX))
print(len(testXunsc))

44509
44509


In [ ]:
pos = lookbackhour - 1


predictiontest = np.zeros(len(testX))
ytestplot = np.zeros(len(testX))

for i in range(0, len(testX)):
    #model.hidden = model.init_hidden()
    predic = model(testX[i])
    #only intereseted in last point as this is the actual predicition
    pred = predic[-1]
    pred= pred.detach().numpy()
    pred = scaler.inverse_transform(pred)
    targetdum = testYunsc[i]
    target = targetdum[-1]
    target = target.detach().numpy()
    predictiontest[i] = np.squeeze(pred)
    ytestplot[i] = np.squeeze(target)
    print(i)

print(predictiontest)   
print(ytestplot)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
testtimes = torch.arange(0, len(testX))
plt.scatter(testtimes, ytestplot, color='b')
plt.scatter(testtimes, predictiontest, color='r')
plt.show()
#plt.savefig(ytest

In [ ]:
def persistance(inputlist, Dstloc):
    output = np.zeros(len(inputlist))
    for i in range(0, len(inputlist)):
        inputbatch = inputlist[i]
        output[i] = inputbatch[-1, 0, Dstloc]
    return(output)

perstest = persistance(testXunsc, 5)

In [ ]:
def CC2(r1, r2):
    return (np.cov(r1,r2)[0][1])/(np.sqrt(np.var(r1)*np.var(r2)))

error = 0
errorpers = 0
MAE = 0
MAEpers = 0

i = 0
for i in range(0, len(predictiontest)):
    error = error + (ytestplot[i] - predictiontest[i])**2
    errorpers = errorpers +  (ytestplot[i] - perstest[i])**2
    MAE = MAE + np.absolute(ytestplot[i] - predictiontest[i])
    MAEpers = MAEpers + np.absolute(ytestplot[i] - perstest[i])
    
MAE = MAE/len(predictiontest)
MAEpers = MAEpers/len(perstest)
    
error = np.sqrt(error / len(predictiontest))
errorpers = np.sqrt(errorpers / len(perstest))


rmse2test = np.sqrt(mse(ytestplot, predictiontest))
rmse2pers = np.sqrt(mse(ytestplot, perstest))

mae2test = mae(ytestplot, predictiontest)
mae2pers = mae(ytestplot, perstest)

Rmodel = CC2(predictiontest, ytestplot)
Rpers = CC2(perstest, ytestplot)

print("RMSE of test data ", forecasthour, " hours in advance forecast: ", error, rmse2test)
print("RMSE of persistance model on test data ", forecasthour, " hours in advance forecast: ", errorpers, rmse2pers)
print("R of test data ", forecasthour, " hours in advance forecast: ", Rmodel)
print("R of persistance model on test data ", forecasthour, " hours in advance forecast: ", Rpers)
print("MAE of test data ", forecasthour, " hours in advance forecast: ", MAE, mae2test)
print("MAE of persistance model on test data ", forecasthour, " hours in advance forecast: ", MAEpers, mae2pers)


In [ ]:
def precision50(forecast, target):
    true_neg = 0
    true_pos = 0
    false_neg = 0
    false_pos = 0
    for i in range(0, len(forecast)):
        if forecast[i] < -50.0 and target[i] < -50.0:
            true_pos = true_pos + 1
        elif forecast[i] < -50.0 and target[i] > -50.0:
            false_pos = false_pos + 1
        elif forecast[i] > -50.0 and target[i] < -50.0:
            false_neg = false_neg + 1
        else:
            true_neg = true_neg + 1
       
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    f1 = (2 * (precision * recall))/(precision + recall)
    
    return precision, recall, f1
    

        

In [ ]:
pres, rec, f1 = precision50(predictiontest, ytestplot)
p_pres, p_rec, p_f1 = precision50(perstest, ytestplot)

In [ ]:

print("50, dataset 1")
print("precision of test data ", forecasthour, " hours in advance forecast: ", pres)
print("precision of persistance model on test data ", forecasthour, " hours in advance forecast: ", p_pres)
print("Recall of test data ", forecasthour, " hours in advance forecast: ", rec)
print("Recall of persistance model on test data ", forecasthour, " hours in advance forecast: ", p_rec)
print("f1 of test data ", forecasthour, " hours in advance forecast: ", f1)
print("f1 of persistance model on test data ", forecasthour, " hours in advance forecast: ", p_f1)

In [ ]:
def precision100(forecast, target):
    true_neg = 0
    true_pos = 0
    false_neg = 0
    false_pos = 0
    for i in range(0, len(forecast)):
        if forecast[i] < -100.0 and target[i] < -100.0:
            true_pos = true_pos + 1
        elif forecast[i] < -100.0 and target[i] > -100.0:
            false_pos = false_pos + 1
        elif forecast[i] > -100.0 and target[i] < -100.0:
            false_neg = false_neg + 1
        else:
            true_neg = true_neg + 1
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)
    f1 = (2 * (precision * recall))/(precision + recall)
    
    return precision, recall, f1

        

In [ ]:
pres, rec, f1 = precision100(predictiontest, ytestplot)
p_pres, p_rec, p_f1 = precision100(perstest, ytestplot)

In [ ]:

print("100, dataset 1")
print("precision of test data ", forecasthour, " hours in advance forecast: ", pres)
print("precision of persistance model on test data ", forecasthour, " hours in advance forecast: ", p_pres)
print("Recall of test data ", forecasthour, " hours in advance forecast: ", rec)
print("Recall of persistance model on test data ", forecasthour, " hours in advance forecast: ", p_rec)
print("f1 of test data ", forecasthour, " hours in advance forecast: ", f1)
print("f1 of persistance model on test data ", forecasthour, " hours in advance forecast: ", p_f1)